In [ ]:


# Simulation parameters
TSim = 100
Ts = 1
N = int(TSim/Ts) + 1

# Path for MV
MVPath = {0: 0, 5: 1, 50: 2, 80: 3, TSim: 3}

t = []
MV = [0]  # Initialisation
MVDelay = [0]  # Initialisation
PV_EBD = []
PV_EFD = []
PV_TRAP = []

# System Parameters
Kp = 1
TLead = 2.5  # Nouveau paramètre
TLag = 5  # Nouveau paramètre
theta = 10  # Délai

for i in range(N):
    t.append(i * Ts)
    SelectPath_RT(MVPath, t, MV)
    
    # Vérification de MV[-2] au début
    if len(MV) < 2:
        MV.append(MV[0])  # Répéter la première valeur si nécessaire
    
    Delay_RT(MV, theta, Ts, MVDelay)
    
    LL_RT(MVDelay, Kp, TLead, TLag, Ts, PV_EBD)
    LL_RT(MVDelay, Kp, TLead, TLag, Ts, PV_EFD, 0, 'EFD')
    LL_RT(MVDelay, Kp, TLead, TLag, Ts, PV_TRAP, 0, 'TRAP')

# **Correction : Ajustement de la taille des listes pour éviter l'erreur**
min_len = min(len(t), len(MV), len(MVDelay))
t = t[:min_len]
MV = MV[:min_len]
MVDelay = MVDelay[:min_len]
PV_EBD = PV_EBD[:min_len]
PV_EFD = PV_EFD[:min_len]
PV_TRAP = PV_TRAP[:min_len]

# Plot results
plt.figure(figsize=(15, 9))

plt.subplot(2, 1, 1)
plt.step(t, MV, 'b-', label='MV', where='post')
plt.step(t, MVDelay, 'b-', linewidth=0.5, label='MV plus delay', where='post')
plt.ylabel('Value of MV')
plt.title('First order plus delay response')
plt.legend(loc='best')
plt.xlim([0, TSim])

plt.subplot(2, 1, 2)
plt.step(t, PV_EBD, 'g-', label='First order response plus delay (EBD)', where='post')
plt.step(t, PV_EFD, 'lime', label='First order response plus delay (EFD)', where='post')
plt.step(t, PV_TRAP, 'springgreen', label='First order response plus delay (TRAP)', where='post')
plt.ylabel('Value of PV')
plt.xlabel('Time [s]')
plt.legend(loc='best')
plt.xlim([0, TSim])

plt.show()

In [ ]:
    #----------------Proportionnal_ Action-----------
    if len(MVP)==0 :
        MVP.append((KC*E[-1]))
    else:
        if method == 'EBD':
            MVP.append(KC*E[-1])
        elif method == 'TRAP':  
            MVP.append(KC*E[-1])        
        else: 
            MVP.append(KC*E[-1])
    
    #-----------------Integral Action---------
    if len(MVI) == 0 :      
        MVI.append((KC*Ts/Ti)*E[-1])    # MV[k] is MV[-1] and MV[k-1] is MV[-2]
    else:
        if method == 'EBD':
            MVI.append(MVI[-1] + ((KC*Ts)/Ti)*(E[-1]))
        elif method == 'TRAP':
            MVI.append(MVI[-1] + (0.5*KC*Ts/Ti)*(E[-1] + E[-2]))
        else:      #default :EBD
            MVI.append(MVI[-1] + (KC*Ts/Ti)*E[-1])

    #------------------Derivative Action-------------
    if len(MVD)==0:
        MVD.append(((KC*Td)/(TFD+Ts))*(E[-1]))
    else:
        if method == 'EBD':
            MVD.append((TFD/(TFD+Ts))*MVD[-1] + ((KC*Td)/(TFD+Ts))*(E[-1] - E[-2])) 
        elif method =='TRAP':
            MVD.append((((TFD-(Ts/2))/(TFD+(Ts/2)))*MVD[-1] + ((KC*Td)/(TFD+(Ts/2)))*(E[-1] - E[-2])))     
        else:  # EBD
            MVD.append((TFD/(TFD+Ts))*MVD[-1] + ((KC*Td)/(TFD+Ts))*(E[-1] - E[-2]))



    #------------Mode manuel + anti-wind up (integrator help)----------
    if Man[-1] == True:
        if ManFF:
            MVI[-1] = MVMan[-1] - MVP[-1] - MVD[-1] 
        else:
            MVI[-1] = MVMan[-1] - MVP[-1] - MVD[-1] - MVFF[-1]

    # -----------------Saturation----------------------
    if (MVP[-1] + MVI[-1] + MVD[-1] + MVFF[-1]) > MVMax:
        MVI[-1] = MVMax - MVP[-1] - MVD[-1] - MVFF[-1]
    elif (MVP[-1] + MVI[-1] + MVD[-1] + MVFF[-1]) < MVMin:
        MVI[-1] = MVMin - MVP[-1] - MVD[-1] - MVFF[-1]

    # Ajout sur MV
    MV.append(MVP[-1] + MVI[-1] + MVD[-1] + MVFF[-1])

